In [ ]:
## ----------------------------------------------------------------------------------------------
# Install Packages - only need to do this once
install.packages(c("rstac", "terra", "RColorBrewer", "getPass", "earthdatalogin"))
library(rhdf5)
library(httr)
library(purrr)
library(stringr)
library(sf)
library(rstac)          # interact with stac catalogs
library(terra)          # work with raster data
library(RColorBrewer)   # color pallets from Color Brewer, a library of colors designed for optimized readability in cartograhy
library(getPass)        # allows us to give the code our password without it printing to the console
library(earthdatalogin) # lets us authenticate to access NASA data

setwd("/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/R_Codes")
source("GEDI_FUN.R")
folder = "/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/Data/L2A/"
AOI = ("/Users/nhunka/Documents/Biomass_Harmonization/Data/Senegal_Workshop/GGNP_Nigeria.gpkg")
AOI_vect = st_read(AOI)
AOI_bbox = st_bbox(AOI_vect)

## ----------------------------------------------------------------------------------------------
# GDAL optimization for working with cloud-optimized data
setGDALconfig("GDAL_DISABLE_READDIR_ON_OPEN", "EMPTY_DIR")
setGDALconfig("CPL_VSIL_CURL_ALLOWED_EXTENSIONS", "TIF") 

## ----------------------------------------------------------------------------------------------
earthdata_username <- getPass("Earthdata User:")
earthdata_password <- getPass("Earthdata Password:")

# note: adding forcemask=FALSE would let you see the values as you type. We intentionally hid that for security of the presentation.


## ----------------------------------------------------------------------------------------------
earthdatalogin::edl_netrc(username = earthdata_username, password = earthdata_password)

## ----------------------------------------------------------------------------------------------
# connect to the stac endpoint we want to query
nasa_stac <- stac("https://cmr.earthdata.nasa.gov/stac/LPCLOUD")

## ----------------------------------------------------------------------------------------------
# We need bbox defined for STAC queries
# minimum longitude, minimum latitude, maximum longitude, and maximum latitude ---> 10 Mile Dunes
bbox = c(AOI_bbox[2],AOI_bbox[4],AOI_bbox[1],AOI_bbox[3])
# Later we'll need the same bbox as a Terra extent object for reading data
extent = ext(c(AOI_bbox[2],AOI_bbox[4],AOI_bbox[1],AOI_bbox[3]))

plot(extent, col="lightblue") #ok, this is perhaps not that

search_hls <- stac_search(
  q = nasa_stac, # The STAC API connection we made earlier
  collections = "GEDI02_A_002", # https://lpdaac.usgs.gov/products/hlss30v002/
  bbox = bbox, #bounding box that we made eariler
  datetime = "2022-06-01T00:00:00Z/2022-07-30T00:00:00Z",  # A closed interval: e.g. "2018-02-12T00:00:00Z/2018-03-18T12:31:12Z" 
  limit = 100 #limits how many results we see
)

results_hls <- post_request(search_hls)
urls<-assets_url(results_hls)
urls<-urls[grep('.h5', urls)]
setwd(folder)

for (url in urls){
  edl_netrc(username, password, "netrc")
  edl_download(url, netrc_path = edl_netrc_path(),cookie_path = edl_cookie_path(), quiet = TRUE)
}

L2A.ls<-list.files(folder, recursive = TRUE, full.names = TRUE)
L2A.ls<-L2A.ls[str_detect(L2A.ls,pattern=".h5")]
L2A.ls

crs    <- CRS("+init=epsg:4326") 
for (h5_file in L2A.ls){
  l = readLevel2A(h5_file)
  try(L2A.m<-getLevel2A(l), silent = TRUE)
  L2A.sp<-SpatialPointsDataFrame(coords=cbind(L2A.m$lon_lowestmode,L2A.m$lat_lowestmode),data=L2A.m,proj4string = crs)
  L2A.sp <- st_as_sf(L2A.sp)
  L2A.sp <- st_intersection(L2A.sp,AOI_vect)
  L2A.sp <- L2A.sp[L2A.sp$quality_flag == 1,]
  L2A.sp <- L2A.sp[L2A.sp$sensitivity > 0.95,]
  st_write(L2A.sp, paste0(folder,gsub(".h5",".gpkg",  basename(h5_file))),overwrite=TRUE)
}